In [ ]:
pip install opencv-python scikit-image


In [ ]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# Paths to input and output videos
input_video_path = '/content/input.mp4'
output_video_path = '/content/REALESR output.mp4'

# Open the video files
input_cap = cv2.VideoCapture(input_video_path)
output_cap = cv2.VideoCapture(output_video_path)

# Check if videos opened successfully
if not input_cap.isOpened():
    print(f"Error: Could not open input video: {input_video_path}")
if not output_cap.isOpened():
    print(f"Error: Could not open output video: {output_video_path}")

if input_cap.isOpened() and output_cap.isOpened():
    # Check total frames (optional validation)
    frame_count = int(min(input_cap.get(cv2.CAP_PROP_FRAME_COUNT), output_cap.get(cv2.CAP_PROP_FRAME_COUNT)))
    print(f"Processing {frame_count} frames...")

    total_psnr = 0
    total_ssim = 0

    for i in range(frame_count):
        ret1, frame1 = input_cap.read()
        ret2, frame2 = output_cap.read()

        if not ret1 or not ret2:
            print(f"Frame read failed at index {i}")
            break

        # Convert to grayscale for SSIM (optional: if you want per-channel SSIM, use color)
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Resize input to output resolution if needed
        if gray1.shape != gray2.shape:
            gray1 = cv2.resize(gray1, (gray2.shape[1], gray2.shape[0]))
            frame1 = cv2.resize(frame1, (frame2.shape[1], frame2.shape[0]))

        total_psnr += psnr(frame2, frame1, data_range=255)
        total_ssim += ssim(gray2, gray1, data_range=255)

    # Average metrics
    if frame_count > 0:
        avg_psnr = total_psnr / frame_count
        avg_ssim = total_ssim / frame_count
        print(f"\nAverage PSNR: {avg_psnr:.2f} dB")
        print(f"Average SSIM: {avg_ssim:.4f}")
    else:
        print("No frames were processed.")

# Release resources
input_cap.release()
output_cap.release()